In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-alignment-of-serine-proteases/iter_aln.faa
/kaggle/input/new-alignment-of-serine-proteases/new_aln.faa
/kaggle/input/new-alignment-of-serine-proteases/new_aln0.faa
/kaggle/input/serprotaln-nocatser/iter_aln.faa
/kaggle/input/serprotaln-nocatser/new_aln.faa


In [2]:
%%capture

!git clone -q https://github.com/spqb/adabmDCApy
!cd adabmDCApy && pip install -q --disable-pip-version-check --root-user-action=ignore .
!pip uninstall -qy --disable-pip-version-check --root-user-action=ignore torch
!pip install -q --disable-pip-version-check --root-user-action=ignore torch==2.4

In [5]:
from adabmDCA.fasta import import_from_fasta, compute_weights, encode_sequence, decode_sequence
import torch

device = 'cuda'

# Compute weights

*Written by Shoichi Yip. Last update: 20 December 2024.*

Sequences are not homogeneously spread in sequence space. This means that highly clustered sequences, which are more similar to each other with respect to the rest of sequences, might impose a strong bias on the training of models. To get rid of this bias, we can perform **reweighting**. This means that each sequence is weighted inversely proportionally to the number of sequence that belong to the same cluster. The cluster is determined by the threshold of similarity that we choose.

In [10]:
dataset_name = 'iter_aln'

The dataset is imported.

In [11]:
d, s = import_from_fasta(f'/kaggle/input/serprotaln-nocatser/{dataset_name}.faa')
se = encode_sequence(s, '-ACDEFGHIKLMNPQRSTVWY')

The dataset is sent to the GPU. Weights are computed.

In [12]:
se_t = torch.tensor(se).to(device)
w = compute_weights(se_t, th=0.8, device=device)

A text file containing the weights is saved.

In [13]:
np.savetxt(f'{dataset_name}_weights.txt', w.cpu().numpy())